In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-p5/sample_submission.csv
/kaggle/input/2021-ml-p5/pca_test.csv
/kaggle/input/2021-ml-p5/pca_train.csv
/kaggle/input/2021-ml-p5/train_label.csv


In [2]:
RANDOM_STATE = 42
file_dir = '/kaggle/input/2021-ml-p5/'
X_train = pd.read_csv(file_dir+'pca_train.csv')
y_train = pd.read_csv(file_dir+'train_label.csv')
X_test = pd.read_csv(file_dir+'pca_test.csv')
sample_submission = pd.read_csv(file_dir+'sample_submission.csv')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

## 데이터 확인

In [4]:
y_train.head()

,id,label
0,0,136
1,1,90
2,2,112
3,3,165
4,4,73


In [5]:
y_train = y_train['label']

In [6]:
y_train

0        136
1         90
2        112
3        165
4         73
        ... 
27666      9
27667    132
27668     18
27669     41
27670    114
Name: label, Length: 27671, dtype: int64

In [7]:
y_train.value_counts()

110    814
17     718
114    692
118    671
117    629
      ... 
42      27
74      27
97      27
142     27
101     26
Name: label, Length: 198, dtype: int64

In [8]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-6.227718,-2.070960,0.674094,-0.742135,0.309149,-4.006772,-0.664624,0.533987,0.024594,-0.195856,...,-2.114192,-0.605770,0.491399,-0.166693,1.289197,-0.403863,0.238731,2.166737,-0.106487,1.486114
1,-6.081077,-1.960213,0.846482,-0.932816,-0.050208,-4.082826,-0.449032,0.647842,0.417809,0.073842,...,-1.000466,-1.175968,0.299093,-1.187048,-0.499101,0.738016,0.632842,0.850430,-2.211427,1.083040
2,-6.394956,-2.954897,1.600033,-0.479463,-1.217065,-3.986472,-0.812934,0.472509,1.536917,1.989765,...,-1.703212,0.713673,0.330699,0.036953,-0.204944,0.930597,-0.693151,-0.572443,0.835598,-0.751483
3,-6.681355,-2.055031,1.144864,-0.517872,-0.168057,-3.929032,-0.718259,0.555827,0.096472,0.724261,...,0.434870,1.571784,-1.266089,1.584173,-0.290301,0.068907,-1.148258,1.904273,0.322937,-0.318100
4,-5.716713,-2.163729,0.877930,-0.605268,0.082372,-4.261466,-0.679734,0.739369,0.490033,0.546437,...,0.626112,-0.435030,2.112485,1.101894,1.746205,-0.445612,-1.518138,0.299177,-0.020655,0.234824


In [9]:
X_train.describe()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
count,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,...,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04,2.767100e+04
mean,-1.533248e-15,-1.045570e-15,1.025509e-15,-7.985694e-16,7.390460e-16,-5.950933e-16,-3.486624e-18,-7.198836e-16,3.920476e-16,1.947414e-16,...,-2.360449e-15,3.235065e-15,-5.880679e-16,-2.777025e-15,2.169796e-15,-7.289813e-16,-9.816359e-16,-1.870829e-15,-3.627079e-15,-3.091500e-15
std,3.714928e+01,1.835446e+01,1.367432e+01,1.172752e+01,7.809187e+00,7.613003e+00,6.279206e+00,5.864037e+00,4.673321e+00,4.575065e+00,...,1.015379e+00,1.014439e+00,1.013780e+00,1.012395e+00,1.012365e+00,1.010778e+00,1.009294e+00,1.008867e+00,1.008407e+00,1.007961e+00
min,-7.990512e+00,-5.337635e+01,-9.835455e+01,-7.036460e+01,-1.117090e+02,-2.414619e+01,-1.239665e+01,-5.413255e+01,-9.599251e+01,-1.180664e+02,...,-4.487069e+00,-3.780968e+00,-6.632077e+00,-4.565285e+00,-6.374972e+00,-5.106658e+00,-3.968695e+00,-4.077860e+00,-4.461192e+00,-4.828997e+00
25%,-6.284841e+00,-2.269306e+00,2.664510e-01,-8.785377e-01,-3.179992e-01,-4.179711e+00,-1.230432e+00,-1.447882e+00,-2.587097e-01,-3.725885e-01,...,-6.798554e-01,-6.832919e-01,-6.739698e-01,-6.691316e-01,-6.726447e-01,-6.777834e-01,-6.795461e-01,-6.746630e-01,-6.714004e-01,-6.858286e-01
50%,-6.125781e+00,-1.883728e+00,7.065141e-01,-5.336844e-01,8.617797e-02,-3.927136e+00,-7.925083e-01,4.530243e-01,1.819773e-01,1.703973e-01,...,4.167662e-03,-4.680592e-03,4.003264e-03,-5.725629e-03,7.176693e-03,-3.743243e-03,7.047510e-03,2.159961e-03,1.666376e-03,-2.281807e-03
75%,-5.937634e+00,-1.464262e+00,1.130377e+00,-9.670580e-02,5.026450e-01,2.093530e+00,-5.368707e-01,6.564108e-01,5.953468e-01,7.488283e-01,...,6.770583e-01,6.880356e-01,6.766032e-01,6.708254e-01,6.861179e-01,6.793633e-01,6.846972e-01,6.913195e-01,6.744505e-01,6.800307e-01
max,2.609487e+02,3.317192e+02,2.944544e+02,7.107089e+02,1.156876e+02,1.488138e+02,6.510218e+01,8.488038e+01,1.020225e+02,1.437308e+02,...,6.281684e+00,4.557646e+00,7.336868e+00,4.643448e+00,6.466784e+00,5.728602e+00,4.315357e+00,4.273075e+00,5.946938e+00,5.983772e+00


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27671 entries, 0 to 27670
Columns: 256 entries, 0 to 255
dtypes: float64(256)
memory usage: 54.0 MB


## standard scaling
> standard scaling이 데이터들의 분포 모양을 맞춰줄 수 있을까?

In [11]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)

X_train_std = std_scaler.transform(X_train)
X_test_std = std_scaler.transform(X_test)

## 하이퍼파라미터 조정없이 확인

In [12]:
def check_score_whitout_params(model, X, y, metric, 
                               test_size=None, 
                               stratify=None,
                               random_state=None):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=stratify)
    
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    print(f'train: {metric(y_train, y_train_pred)}')
    print(f'test: {metric(y_test, y_test_pred)}')

### LDA

#### standard scaling -> X

In [13]:
check_score_whitout_params(LinearDiscriminantAnalysis(),
                           X_train, y_train, 
                           accuracy_score, 
                           random_state=RANDOM_STATE)

train: 0.6868404568014262
test: 0.5897658282740676


#### standard scaling -> O

In [14]:
check_score_whitout_params(LinearDiscriminantAnalysis(),
                           X_train_std, y_train, 
                           accuracy_score, 
                           random_state=RANDOM_STATE)

train: 0.6868404568014262
test: 0.5897658282740676


### QDA
> over fitting??

#### standard scaling -> X

In [15]:
check_score_whitout_params(QuadraticDiscriminantAnalysis(),
                           X_train, y_train, 
                           accuracy_score, 
                           random_state=RANDOM_STATE)

/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


train: 0.9967715511010456
test: 0.3431627638045678


#### standard scaling -> O

In [16]:
check_score_whitout_params(QuadraticDiscriminantAnalysis(),
                           X_train_std, y_train, 
                           accuracy_score, 
                           random_state=RANDOM_STATE)

/opt/conda/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


train: 0.9205416084421529
test: 0.3431627638045678


### standard scaling 한 데이터로 LDA 이용 제출해보기

In [17]:
cld_std=LinearDiscriminantAnalysis(store_covariance=True)
cld_std.fit(X_train_std, y_train)

y_test_std_pred = cld_std.predict(X_test_std)
sample_submission['label'] = y_test_std_pred

In [18]:
sample_submission.to_csv('submission_LDA_std_no_params.csv', index=False)